# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
weather_df = pd.read_csv('weather.csv')
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,kapaa,90,US,1584493448,94,22.08,-159.32,75.2,6.93
1,1,carmen,5,MX,1584493448,78,18.63,-91.83,80.6,8.05
2,2,yellowknife,75,CA,1584493292,70,62.46,-114.35,-4.0,5.82
3,3,bredasdorp,12,ZA,1584493347,77,-34.53,20.04,66.2,8.05
4,4,nelson bay,20,AU,1584493448,60,-32.72,152.15,77.0,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)
locations = weather_df[['Lat','Lng']]
weights = weather_df['Humidity']

In [26]:
humidity_map = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations,weights=weights)
heatmap.max_intensity = max(weights)
humidity_map.add_layer(heatmap)

humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#Ideal weather is max temp between 70 and 80 degrees, wind speed less than 10 mph, and zero cloudiness
condition1 = weather_df['Max Temp']>=70
condition2 = weather_df['Max Temp']<=80
condition3 = weather_df['Wind Speed']<=10
condition4 = weather_df['Cloudiness'] == 0
narrowed_city_df = weather_df[(condition1&condition2&condition3&condition4)]
narrowed_city_df = narrowed_city_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = narrowed_city_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
136,136,tougue,0,GN,1584493470,18,11.47,-11.60,72.91,3.11,
162,162,gunnedah,0,AU,1584493443,46,-30.98,150.25,73.40,5.82,
171,171,vadlapudi,0,IN,1584493475,91,14.32,79.80,72.18,6.38,
190,190,quatre cocos,0,MU,1584493479,77,-20.21,57.76,78.91,4.00,
253,253,sharjah,0,AE,1584493489,72,25.36,55.40,78.01,2.24,


In [29]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "lat,lng",
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

In [30]:
for index, row in hotel_df.iterrows():

    # get lat and lng from hotel_df
    city = row['City']
    coords = f"{row['Lat']},{row['Lng']}"

    # add keyword to params dict
    params['location'] = coords

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 136: tougue.
Missing field/result... skipping.
------------
Retrieving Results for Index 162: gunnedah.
------------
Retrieving Results for Index 171: vadlapudi.
------------
Retrieving Results for Index 190: quatre cocos.
------------
Retrieving Results for Index 253: sharjah.
------------
Retrieving Results for Index 254: mattru.
------------
Retrieving Results for Index 255: dwarka.
------------
Retrieving Results for Index 282: chiang rai.
------------
Retrieving Results for Index 356: airai.
------------
Retrieving Results for Index 425: kaeo.
------------
Retrieving Results for Index 489: kovur.
------------
Retrieving Results for Index 500: antalaha.
------------


In [31]:
condition = hotel_df['Hotel Name'] != ''
hotel_df = hotel_df[condition]
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
162,162,gunnedah,0,AU,1584493443,46,-30.977596,150.247518,73.40,5.82,Railway Hotel
171,171,vadlapudi,0,IN,1584493475,91,14.345120,79.858820,72.18,6.38,OYO 45909 Kmc Residency
190,190,quatre cocos,0,MU,1584493479,77,-20.203514,57.772807,78.91,4.00,Horisun
253,253,sharjah,0,AE,1584493489,72,25.361918,55.391703,78.01,2.24,فندق سنتر الشارقة Center Hotel Sharjah
254,254,mattru,0,SL,1584493489,85,7.933333,-11.766667,79.41,3.42,Bo inn Hotel
255,255,dwarka,0,IN,1584493489,61,22.238799,68.970326,78.19,8.63,Hotel Shree Vallabh
282,282,chiang rai,0,TH,1584493494,82,19.908538,99.829474,70.00,1.43,Tongsiam
356,356,airai,0,TL,1584493506,64,-8.984578,125.503633,73.18,4.74,Restaurant Gabmenis
425,425,kaeo,0,NZ,1584493518,48,-35.054260,173.740702,71.01,1.99,Pacific Harbour Lodge
489,489,kovur,0,IN,1584493528,91,14.458604,79.992023,72.18,6.38,Sri Vinayaka Lodge


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df 

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
162,162,gunnedah,0,AU,1584493443,46,-30.977596,150.247518,73.40,5.82,Railway Hotel
171,171,vadlapudi,0,IN,1584493475,91,14.345120,79.858820,72.18,6.38,OYO 45909 Kmc Residency
190,190,quatre cocos,0,MU,1584493479,77,-20.203514,57.772807,78.91,4.00,Horisun
253,253,sharjah,0,AE,1584493489,72,25.361918,55.391703,78.01,2.24,فندق سنتر الشارقة Center Hotel Sharjah
254,254,mattru,0,SL,1584493489,85,7.933333,-11.766667,79.41,3.42,Bo inn Hotel
255,255,dwarka,0,IN,1584493489,61,22.238799,68.970326,78.19,8.63,Hotel Shree Vallabh
282,282,chiang rai,0,TH,1584493494,82,19.908538,99.829474,70.00,1.43,Tongsiam
356,356,airai,0,TL,1584493506,64,-8.984578,125.503633,73.18,4.74,Restaurant Gabmenis
425,425,kaeo,0,NZ,1584493518,48,-35.054260,173.740702,71.01,1.99,Pacific Harbour Lodge
489,489,kovur,0,IN,1584493528,91,14.458604,79.992023,72.18,6.38,Sri Vinayaka Lodge


In [33]:
# Add marker layer ontop of heat map
hotel_map = gmaps.figure()
hotel_map.add_layer(gmaps.marker_layer(locations,info_box_content=hotel_info,display_info_box=True))
# Display Map
hotel_map

Figure(layout=FigureLayout(height='420px'))